In [ ]:
# Flask app
def create_app():
    from flask import Flask, request, jsonify
    
    app = Flask(__name__)
    
    model: LogisticRegression = joblib.load("best_model.pkl")  # Load the trained model
    
    @app.route("/score", methods=["POST"])
    def score_endpoint():
        data = request.json
        text = data.get("text", "")
        threshold = float(data.get("threshold", 0.5))
        
        prediction, propensity = score(text, model, threshold)
        return jsonify({"prediction": int(prediction), "propensity": propensity})
    
    return app

# Integration test for Flask app
def test_flask():
    import os
    import time
    import requests
    import subprocess
    
    # Start the Flask app
    process = subprocess.Popen(["python", "app.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(2)  # Wait for server to start
    
    url = "http://127.0.0.1:5000/score"
    data = {"text": "This is spam!", "threshold": 0.5}
    response = requests.post(url, json=data)
    
    assert response.status_code == 200, "Flask endpoint should return 200."
    json_response = response.json()
    assert "prediction" in json_response and "propensity" in json_response, "Response should contain prediction and propensity."
    
    # Stop the Flask app
    process.terminate()
    process.wait()

# Generate coverage report
if __name__ == "__main__":
    import os
    os.system("pytest --cov=score --cov=app --cov-report=term-missing > coverage.txt")
